In [1]:
"""
Convolutional Neuroscience
Accademic year 2019-2020
Homework 3

Author: Tommaso Tabarelli
Period: december 2019
"""

# Importing libraries

import argparse
import torch
import json
import re
import numpy as np
from torch.utils.data import Dataset, DataLoader
from functools import reduce
from torch import optim, nn
from network import Network, train_batch
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path

# Importing packages to implement word2vec
from torch.autograd import Variable
import torch.functional as F
import torch.nn.functional as F

### Implementing word2vec

Following the instructions at the following link: https://towardsdatascience.com/implementing-word2vec-in-pytorch-skip-gram-model-e6bae040d2fb

In [2]:
### Load data
text = open("Picture_of_Dorian_Gray.txt", 'r').read()

# Removing titles
text = re.split('\n{7}', text)[1]

# Lowering all text
text = text.lower()

sentences = re.split('[\.,!?;:]', text)

vocabulary = []
all_words = re.split("[\.,!?;:\n -\'-]", text)
for word in all_words:
    if word not in vocabulary:
        vocabulary.append(word)

vocabulary.sort()

# Removing the first 3 elements since they are numbers or empty string
vocabulary.pop(0)
vocabulary.pop(0)
vocabulary.pop(0)

print(len(text.split(".")))
print(len(sentences))
#print(vocabulary)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

#print(word2idx.keys())

print(sentences[0])
for sentence in sentences:
    # saving index
    index = sentences.index(sentence)
    sentence = re.split("[\.,!?;:\n -\'-]", sentence)
    
    # Removing by hand ALL the unwanted characters
    for i in range(sentence.count('')):
        sentence.pop(sentence.index(''))
    for i in range(sentence.count(' ')):
        sentence.pop(sentence.index(" "))
    for i in range(sentence.count('\n')):
        sentence.pop(sentence.index("\n"))
    if ("152" in sentence):
        sentence.pop(sentence.index("152"))
    if ("1820" in sentence):
        sentence.pop(sentence.index("1820"))
    # Overwriting the sentence with a word-split one
    sentences[index] = sentence
print(sentences[0])

5746
12519
the studio was filled with the rich odour of roses
['the', 'studio', 'was', 'filled', 'with', 'the', 'rich', 'odour', 'of', 'roses']


### Creating "context" environment to then train the net

In [3]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in sentences:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make sure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [4]:
len(idx_pairs)

248350

## Starting to define the Neural Network structure to embed words

### Input layer

It should be as large as the vocabulary.

In [10]:
def get_input_layer(word_idx):
    x = torch.zeros(len(vocabulary)).float()
    x[word_idx] = 1.0
    return x

### Hidden layers

It is chosen to be only 1 hidden layer and it depends on the embedding dimension (arbitraryli chosen).

W1 is the weight matrix. It has dimensions: [embedding_dims, vocabulary_size]

There is no activation function — just plain matrix multiplication. (This will be clearer during training)

In [11]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, len(vocabulary)).float(), requires_grad=True)

### Output layer

In [12]:
W2 = Variable(torch.randn(len(vocabulary), embedding_dims).float(), requires_grad=True)

### Training

In [96]:
num_epochs = 101
learning_rate = 0.001

# Define optimizer
optimizer = optim.Adam(my_net.parameters(), weight_decay=5e-4)

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:

        # Forward pass
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
        

        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
        
        #optimizer.step()
        
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 9.346418637197958
Loss at epo 10: 7.211674061661062
Loss at epo 20: 6.778484885877758
Loss at epo 30: 6.574319957424175
Loss at epo 40: 6.451177387590353
Loss at epo 50: 6.367360884102869
Loss at epo 60: 6.305997703103249
Loss at epo 70: 6.258726537012349
Loss at epo 80: 6.22101781125469
Loss at epo 90: 6.190239186260286
Loss at epo 100: 6.164635735624557


In [97]:
W2.type()

'torch.FloatTensor'

In [106]:
torch.save(W1, "W1_weights")
torch.save(W2, "W2_weights")

In [103]:
print(W2.shape)

torch.Size([6779, 5])


### Loading saved parameters for the embedding

In [5]:
W1_try = torch.load("W1_weights")
W2_try = torch.load("W2_weights")
print(W1_try)
print(W2_try)
print(torch.max(W1_try).item())
print(torch.max(W2_try).item())

tensor([[-0.6536, -0.2067, -0.2938,  ..., -0.3121, -0.6741,  1.1771],
        [ 0.2630,  1.2787,  1.1708,  ..., -0.9615,  0.1497,  1.2112],
        [ 0.5051,  0.5456, -0.4282,  ..., -1.2316,  0.3074,  0.5471],
        [-0.2665, -0.3556, -0.5891,  ...,  0.3163,  1.4133,  1.0261],
        [ 0.4310,  1.6727,  1.3909,  ...,  1.3273,  0.9603,  0.7538]],
       requires_grad=True)
tensor([[-4.7673,  0.7313,  1.7859, -1.6416,  3.9699],
        [ 0.9328, -0.2321,  0.7114,  0.2274, -0.5491],
        [-0.0083, -0.1352,  0.4440, -0.7173, -0.1907],
        ...,
        [ 0.1161, -1.2554,  1.0307, -1.0508, -0.2169],
        [ 1.4459, -1.2231, -0.3555,  0.4521,  1.1497],
        [-0.5359,  0.0490,  0.0687, -0.1203, -1.4884]], requires_grad=True)
3.3844878673553467
4.363911151885986


Taking W2 as the word representation because it depends on context! In other words, I think the center is less biased term, and what I need is to be able to predict words of a given text/style/piece of paper; for this reason context representation is better.

## Using LSTM network

Our network is already well implemented and it uses a LSTM layer.

In [9]:
# Defining network class

class Network(nn.Module):
    
    def __init__(self, input_size, hidden_units, layers_num, dropout_prob=0):
        # Call the parent init function (required!)
        super().__init__()
        # Define recurrent layer
        self.rnn = nn.LSTM(input_size=input_size, 
                        hidden_size=hidden_units,
                        num_layers=layers_num,
                        dropout=dropout_prob,
                        batch_first=True)
        # Define output layer
        self.out = nn.Linear(hidden_units, input_size)

    def forward(self, x, state=None):
        # LSTM
        x, rnn_state = self.rnn(x, state)
        # Linear layer
        x = self.out(x)
        return x, rnn_state
    



def train_batch(net, batch_repr, loss_fn, optimizer):

    ### Prepare network input and labels
    # Get the labels (the last letter of each sequence)
    labels_repr = batch_repr[:, -1, :]
    
    # This is no more useful
    #labels_numbers = labels_onehot.argmax(dim=1)
    
    # Remove the labels from the input tensor
    net_input = batch_repr[:, :-1, :]
    # batch_repr.shape =   [50, 100, 5]
    # labels_repr.shape =  [50, 5]
    # net_input.shape =      [50, 99, 5]

    ### Forward pass
    # Eventually clear previous recorded gradients
    optimizer.zero_grad()
    # Forward pass
    net_out, _ = net(net_input)

    ### Update network
    # Evaluate loss only for last output
    loss = loss_fn(net_out[:, -1, :], labels_repr)
    # Backward pass
    loss.backward()
    # Update
    optimizer.step()
    # Return average batch loss
    return float(loss.data)

In [10]:
class word_Dataset(Dataset):
    
    def __init__(self, filepath, transform=None):

        ### Load data
        text = open("Picture_of_Dorian_Gray.txt", 'r').read()

        # Removing titles
        text = re.split('\n{7}', text)[1]

        # Lowering all text
        text = text.lower()

        sentences = re.split('[\.,!?;:]', text)

        vocabulary = []
        all_words = re.split("[\.,!?;:\n -\'-]", text)
        for word in all_words:
            if word not in vocabulary:
                vocabulary.append(word)

        vocabulary.sort()

        # Removing the first 3 elements since they are numbers or empty string
        vocabulary.pop(0)
        vocabulary.pop(0)
        vocabulary.pop(0)

        word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
        idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

        for sentence in sentences:
            # saving index
            index = sentences.index(sentence)
            
            sentence = re.split("[\.,!?;:\n -\'-]", sentence)

            # Removing by hand ALL the unwanted characters
            for i in range(sentence.count('')):
                sentence.pop(sentence.index(''))
            for i in range(sentence.count(' ')):
                sentence.pop(sentence.index(" "))
            for i in range(sentence.count('\n')):
                sentence.pop(sentence.index("\n"))
            if ("152" in sentence):
                sentence.pop(sentence.index("152"))
            if ("1820" in sentence):
                sentence.pop(sentence.index("1820"))
            # Overwriting the sentence with a word-split one
            sentences[index] = sentence


        # Removing too short sentences (which are not considered sentences...)
        large_sentences = []
        
        for sent in sentences:
            if (len(sent)>5):
                large_sentences.append(sent)
        
        self.sentences = large_sentences
        self.vocabulary = vocabulary
        self.word_to_index = word2idx
        self.index_to_word = idx2word
        self.transform = transform
        
    def __len__(self):
        return len(self.sentences)
        
    def __getitem__(self, idx):
        # Get sentence text
        text = self.sentences[idx]
        # Encode with numbers
        encoded = encode_text(self.word_to_index, text)

        # Create sample
        sample = {'text': text, 'encoded': encoded}
        # Transform (if defined)
        if self.transform:
            sample = self.transform(sample)
        return sample


def encode_text(word_to_index, text):
    encoded = [word_to_index[w] for w in text]
    return encoded


def decode_text(index_to_word, encoded):
    text = [index_to_word[c] for c in encoded]
    # Building proper string from a list of strings (concatenating them)
    text = reduce(lambda s1, s2: s1 + " " + s2, text)
    return text


class RandomCrop():
    
    def __init__(self, crop_len):
        self.crop_len = crop_len
        
    def __call__(self, sample):
        text = sample['text']
        encoded = sample['encoded']
        # Randomly choose an index
        tot_words = len(text)
        if ((tot_words - self.crop_len) > 0):
            start_idx = np.random.randint(0, tot_words - self.crop_len)
        # if no operations can be done to sort the sentence
        else:
            return sample
        end_idx = start_idx + self.crop_len
        return {**sample,
            'text': text[start_idx: end_idx],
            'encoded': encoded[start_idx: end_idx]}


def create_W2V_representation(encoded, representation):
    # Create one hot matrix
    #    representation is a matrix with representation row-wise, so
    #     len(representation[0]) gives the dimension of a word
    encoded_repr = np.zeros([len(encoded), len(representation[0])])
    tot_words = len(encoded)
    # Placing words vectors at the respective position of the letters: "encoded" indeed have numbers
    # that encode the words, and here it is used as index dimension
    encoded_repr[:, :] = representation.detach().numpy()[encoded]
    return encoded_repr


class W2V_Encoder():
    
    def __init__(self, W2V_representation):
        self.representation = W2V_representation
        
    def __call__(self, sample):
        # Load encoded text with numbers
        encoded = np.array(sample['encoded'])
        # Create  representation matrix
        encoded_repr = create_W2V_representation(encoded, self.representation)
        return {**sample,
            'encoded_repr': encoded_repr}


class ToTensor():
    
    def __call__(self, sample):
        # Convert one hot encoded text to pytorch tensor
        encoded_repr = torch.tensor(sample['encoded_repr']).float()
        return {'encoded_repr': encoded_repr}

### Checking the sentences have at least 5 words, so to have crop_len = 5

In [9]:
crop_len = 5

trans = transforms.Compose([RandomCrop(crop_len),
                        W2V_Encoder(W2_try),
                        ToTensor()
                        ])

dataset = word_Dataset(filepath="Picture_of_Dorian_Gray.txt", 
                        transform=trans)

In [10]:
my_sentences = []
lenghts = []
for sent in dataset.sentences:
    #print(sent)
    lenghts.append(len(sent))
print(min(lenghts))

6


## Defining network

In [11]:
embedding_dims = 5
hidden_units = 500
num_layers = 2
batchsize = 20

my_net = Network(embedding_dims, hidden_units, num_layers, dropout_prob=0.3)

## Training network

In [127]:
#%% Train network

#%% Check device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Selected device:', device)

# Define Dataloader
dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True, num_workers=1)
# Define optimizer
optimizer = optim.Adam(my_net.parameters(), weight_decay=5e-4)

# Define loss function: using MSE since now the words are represented as vectors
loss_fn = nn.MSELoss()

num_epochs = 100

# Start training
for epoch in range(num_epochs):
    print('##################################')
    print('## EPOCH %d' % (epoch + 1))
    print('##################################')
    # Iterate batches
    for batch_sample in dataloader:
        # Extract batch
        batch_ = batch_sample['encoded_repr'].to(device)
        # Update network
        batch_loss = train_batch(my_net, batch_, loss_fn, optimizer)
        print('\t Training loss (single batch):', batch_loss)


Selected device: cpu
##################################
## EPOCH 1
##################################
	 Training loss (single batch): 3.8221375942230225
	 Training loss (single batch): 4.3480072021484375
	 Training loss (single batch): 3.2949044704437256
	 Training loss (single batch): 2.896714448928833
	 Training loss (single batch): 2.490509510040283
	 Training loss (single batch): 1.2603501081466675
	 Training loss (single batch): 1.8766249418258667
	 Training loss (single batch): 1.8936928510665894
	 Training loss (single batch): 1.8663142919540405
	 Training loss (single batch): 1.3003733158111572
	 Training loss (single batch): 1.8950037956237793
	 Training loss (single batch): 1.5635899305343628
	 Training loss (single batch): 1.4965094327926636
	 Training loss (single batch): 1.605830430984497
	 Training loss (single batch): 1.665558099746704
	 Training loss (single batch): 1.3614592552185059
	 Training loss (single batch): 1.6390130519866943
	 Training loss (single batch): 1.8

	 Training loss (single batch): 1.4891713857650757
	 Training loss (single batch): 1.3496159315109253
	 Training loss (single batch): 1.1127424240112305
	 Training loss (single batch): 1.2229430675506592
	 Training loss (single batch): 2.049783945083618
	 Training loss (single batch): 1.5789822340011597
	 Training loss (single batch): 1.2520272731781006
	 Training loss (single batch): 1.0087761878967285
	 Training loss (single batch): 1.2296509742736816
	 Training loss (single batch): 1.4132428169250488
	 Training loss (single batch): 1.522263765335083
	 Training loss (single batch): 1.7031246423721313
	 Training loss (single batch): 1.5257399082183838
	 Training loss (single batch): 1.3766995668411255
	 Training loss (single batch): 1.5454668998718262
	 Training loss (single batch): 1.1475512981414795
	 Training loss (single batch): 1.228161334991455
	 Training loss (single batch): 1.3872756958007812
	 Training loss (single batch): 1.612813949584961
	 Training loss (single batch): 1.8

	 Training loss (single batch): 1.1998635530471802
	 Training loss (single batch): 1.1155022382736206
	 Training loss (single batch): 1.5679799318313599
	 Training loss (single batch): 1.2221055030822754
	 Training loss (single batch): 1.6225379705429077
	 Training loss (single batch): 1.5927387475967407
	 Training loss (single batch): 1.1203464269638062
	 Training loss (single batch): 1.4076415300369263
	 Training loss (single batch): 1.0040093660354614
	 Training loss (single batch): 1.5664825439453125
	 Training loss (single batch): 1.1402521133422852
	 Training loss (single batch): 1.2199534177780151
	 Training loss (single batch): 1.793369174003601
	 Training loss (single batch): 1.4587093591690063
	 Training loss (single batch): 1.3269951343536377
	 Training loss (single batch): 1.4837627410888672
	 Training loss (single batch): 1.234573245048523
	 Training loss (single batch): 1.4877716302871704
	 Training loss (single batch): 1.3176504373550415
	 Training loss (single batch): 1

	 Training loss (single batch): 1.3515899181365967
	 Training loss (single batch): 2.145787000656128
	 Training loss (single batch): 1.8690464496612549
	 Training loss (single batch): 1.5156370401382446
	 Training loss (single batch): 1.2033493518829346
	 Training loss (single batch): 1.0312904119491577
	 Training loss (single batch): 1.0394785404205322
	 Training loss (single batch): 1.5499213933944702
	 Training loss (single batch): 1.4476556777954102
	 Training loss (single batch): 1.0836308002471924
	 Training loss (single batch): 1.0129491090774536
	 Training loss (single batch): 1.340132474899292
	 Training loss (single batch): 1.8998738527297974
	 Training loss (single batch): 1.4201709032058716
	 Training loss (single batch): 1.2257113456726074
	 Training loss (single batch): 1.3299803733825684
	 Training loss (single batch): 1.2208009958267212
	 Training loss (single batch): 1.282057285308838
	 Training loss (single batch): 1.322852373123169
	 Training loss (single batch): 0.9

	 Training loss (single batch): 1.2509607076644897
	 Training loss (single batch): 1.4111363887786865
	 Training loss (single batch): 1.3481616973876953
	 Training loss (single batch): 0.9493928551673889
	 Training loss (single batch): 1.3285434246063232
	 Training loss (single batch): 1.3623472452163696
	 Training loss (single batch): 1.4261010885238647
	 Training loss (single batch): 1.0543949604034424
	 Training loss (single batch): 1.642903447151184
	 Training loss (single batch): 1.3596391677856445
	 Training loss (single batch): 1.858802318572998
	 Training loss (single batch): 1.5170643329620361
	 Training loss (single batch): 1.4027868509292603
	 Training loss (single batch): 1.2103443145751953
	 Training loss (single batch): 1.2561451196670532
	 Training loss (single batch): 1.3040435314178467
	 Training loss (single batch): 1.2727817296981812
	 Training loss (single batch): 1.4239567518234253
	 Training loss (single batch): 1.5195070505142212
	 Training loss (single batch): 1

...

	 Training loss (single batch): 0.9976332783699036
	 Training loss (single batch): 1.2763599157333374
	 Training loss (single batch): 1.0779597759246826
	 Training loss (single batch): 1.2553049325942993
	 Training loss (single batch): 1.4102733135223389
	 Training loss (single batch): 1.0876948833465576
	 Training loss (single batch): 1.418114185333252
	 Training loss (single batch): 1.5859321355819702
	 Training loss (single batch): 0.9674158096313477
	 Training loss (single batch): 1.2174506187438965
	 Training loss (single batch): 1.8446677923202515
	 Training loss (single batch): 1.4392290115356445
	 Training loss (single batch): 1.8030732870101929
	 Training loss (single batch): 1.3520504236221313
	 Training loss (single batch): 1.1530784368515015
	 Training loss (single batch): 1.1120349168777466
	 Training loss (single batch): 1.1104422807693481
	 Training loss (single batch): 1.0262442827224731
	 Training loss (single batch): 1.1588208675384521
	 Training loss (single batch): 

	 Training loss (single batch): 1.1825789213180542
	 Training loss (single batch): 1.6606245040893555
	 Training loss (single batch): 1.3071460723876953
	 Training loss (single batch): 0.958549439907074
	 Training loss (single batch): 1.1698408126831055
	 Training loss (single batch): 1.4297581911087036
	 Training loss (single batch): 1.3579896688461304
	 Training loss (single batch): 0.9409142136573792
	 Training loss (single batch): 0.6701403260231018
	 Training loss (single batch): 1.2375308275222778
	 Training loss (single batch): 0.8183053135871887
	 Training loss (single batch): 0.9118518233299255
	 Training loss (single batch): 1.602758526802063
	 Training loss (single batch): 1.3224725723266602
	 Training loss (single batch): 1.002163052558899
	 Training loss (single batch): 1.1509971618652344
	 Training loss (single batch): 1.211553692817688
	 Training loss (single batch): 1.6206371784210205
	 Training loss (single batch): 1.026116967201233
	 Training loss (single batch): 1.20

	 Training loss (single batch): 1.169464349746704
	 Training loss (single batch): 1.3207274675369263
	 Training loss (single batch): 0.8349696397781372
	 Training loss (single batch): 1.7450125217437744
	 Training loss (single batch): 1.2664909362792969
	 Training loss (single batch): 1.049789309501648
	 Training loss (single batch): 1.5554327964782715
	 Training loss (single batch): 1.0762830972671509
	 Training loss (single batch): 1.2444804906845093
	 Training loss (single batch): 0.8567343950271606
	 Training loss (single batch): 1.0318994522094727
	 Training loss (single batch): 1.2363078594207764
	 Training loss (single batch): 1.072996735572815
	 Training loss (single batch): 0.9329531788825989
	 Training loss (single batch): 1.64449942111969
	 Training loss (single batch): 1.2791904211044312
	 Training loss (single batch): 1.361505389213562
	 Training loss (single batch): 1.473666787147522
	 Training loss (single batch): 1.3812545537948608
	 Training loss (single batch): 1.4743

	 Training loss (single batch): 0.7092276811599731
	 Training loss (single batch): 1.1804310083389282
	 Training loss (single batch): 1.179866909980774
	 Training loss (single batch): 1.149519920349121
	 Training loss (single batch): 1.2726030349731445
	 Training loss (single batch): 1.1411170959472656
	 Training loss (single batch): 1.1647638082504272
	 Training loss (single batch): 1.1580708026885986
	 Training loss (single batch): 1.1781364679336548
	 Training loss (single batch): 1.0193777084350586
	 Training loss (single batch): 1.2907243967056274
	 Training loss (single batch): 0.6156530380249023
	 Training loss (single batch): 1.1302152872085571
	 Training loss (single batch): 0.7739677429199219
	 Training loss (single batch): 1.509239912033081
	 Training loss (single batch): 0.9679361581802368
	 Training loss (single batch): 1.2684799432754517
	 Training loss (single batch): 1.0101829767227173
	 Training loss (single batch): 1.6549524068832397
	 Training loss (single batch): 1.

NameError: name 'args' is not defined

In [129]:
embedding_dims = 5
hidden_units = 500
num_layers = 2
batchsize = 20
num_epochs = 100
crop_len = 5

training_args = {"embedding_dims": 5,
                "hidden_units": 500,
                "num_layers": 2,
                "batchsize": 20,
                "num_epochs": 100,
                "crop_len": 5}

In [130]:
### Save all needed parameters
# Create output dir`
out_dir = Path("model_W2V")
out_dir.mkdir(parents=True, exist_ok=True)
# Save network parameters
torch.save(my_net.state_dict(), out_dir / 'net_params.pth')
# Save training parameters
with open(out_dir / 'training_args.json', 'w') as f:
    json.dump(training_args, f, indent=4)
# Save encoder dictionary
with open(out_dir / 'char_to_number.json', 'w') as f:
    json.dump(dataset.word_to_index, f, indent=4)

## Loading network and trying to predict

In [15]:
# Loading the word representation
W1_try = torch.load("W1_weights")
W2_try = torch.load("W2_weights")

model_dir = Path('model_W2V')

training_args = json.load(open(model_dir / 'training_args.json'))

#%% Initialize network
my_net = Network(training_args["embedding_dims"],
                 training_args["hidden_units"],
                 training_args["num_layers"],
                 dropout_prob=0.3)

#%% Load network trained parameters
my_net.load_state_dict(torch.load(model_dir / 'net_params.pth', map_location='cpu'))
my_net.eval() # Evaluation mode (e.g. disable dropout)

chapter_seed = "dorian was at his house"
chapter_seed = re.split('[ ]', chapter_seed)

word_to_number = json.load(open(model_dir / 'word_to_number.json'))
number_to_word = {k: v for k,v in enumerate(word_to_number)}

np_repr = W2_try.detach().numpy()

#%% Find initial state of the RNN
with torch.no_grad():
    # Encode seed
    seed_encoded = encode_text(word_to_number, chapter_seed)
    
    # Vector representation
    seed_repr = create_W2V_representation(seed_encoded, W2_try)
    # To tensor
    seed_repr = torch.tensor(seed_repr).float()
    # Add batch axis
    #seed_repr = seed_repr.unsqueeze(0)
        
    # Forward pass
    net_out, net_state = my_net(seed_repr[0].unsqueeze(0).unsqueeze(0))
    
    for word in seed_repr[1:]:
        word.unsqueeze(0)
        net_out, net_state = my_net(word.unsqueeze(0).unsqueeze(0), net_state)
        
    word_vector = net_out.numpy()[0,0,:]
    
    # Looking for the minimum distances between the guessed representation and the words
    next_word_encoded = np.argmin(np.sqrt(np.sum((np_repr - word_vector)**2, axis=1)))
        
    # Get the most probable last output index
    #next_word_encoded = net_out[:, -1, :].argmax().item()
    
    # Print the seed words
    for word in chapter_seed:
        print(word, end=' ', flush=True)
    
    next_word = number_to_word[next_word_encoded]
    print(next_word, end=' ', flush=True)

#%% Generate chapter
tot_word_count = 0
while True:
    with torch.no_grad(): # No need to track the gradients
        # The new network input is the one hot encoding of the last chosen word
        
        net_input = W2_try[next_word_encoded].unsqueeze(0).unsqueeze(0)
        
        # Forward pass
        net_out, net_state = my_net(net_input, net_state)
        
        word_vector = net_out.numpy()[0,0,:]
    
        # Looking for the minimum distances between the guessed representation and the words
        #  and choosing using a softmax
        distances = np.sqrt(np.sum((np_repr - word_vector)**2, axis=1))
        distrib = np.array(nn.functional.softmax(torch.from_numpy(1/distances), dim=-1))
        #next_char_encoded = np.random.choice(len(distrib.ravel()), size=1, p=distrib.ravel())[0]
        next_word_encoded = np.random.choice(len(distrib.ravel()), size=1, p=distrib.ravel())[0]
        
        # Looking for the minimum distances between the guessed representation and the words
        #next_word_encoded = np.argmin(np.sqrt(np.sum((np_repr - word_vector)**2, axis=1)))
    
        next_word = number_to_word[next_word_encoded]
        
        print(next_word, end=' ', flush=True)
        # Count total letters
        tot_word_count += 1
        # Break if 20 lines or 300 words
        if tot_word_count > 300:
            break

dorian was at his house by me players dutch book apple siphons yesterday footed thief movements superstitions amarre wine camillus pebbles pins searched widening temperaments shield moon momentary intentions christian spectators consumption nose untouched isotta see estimate hooded oratory crushed lock cedar illustrated point sovereign recovered stamp dreadful infatuation disk civil christened verities paint devoted take portico beasts eighty jacobean lash intellects droop flowering scolding renewed disappointment gable stock came furry lateness called crumpled down same loses got cheated tussore head cruel buzzed awakened frontleted parts whatever between other nuisance trampling bedside culture happen present sinner fallen grass emperor arrangement feed omnibus lurid bamboo sup consequence tapped demons proposing shaken monotony seizing shaped slander girl unpardonable audace superficial twelve bamboo lives and passion faster schoolgirl kissed sulkily wardour curiously curiosity rich